In [1]:
spark

StatementMeta(, , , Waiting, )

In [2]:
## Necessary Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import *

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 8, Finished, Available)

### Loading Intermediate Data

Here we load the intermediate data which includes 2 datasets: `Comments` and `Submissions`. These datasets include all the Reddit posts between September 2021 and April 2022 (The approximate length of the College Basketball Season). No additional cleaning has already been done on this data 

In [3]:
workspace_default_storage_account = "aml6214727930"
workspace_default_container = "azureml-blobstore-6653633b-3460-4381-9199-d9e0f368353c"

workspace_wasbs_base_url = (
    f"wasbs://{workspace_default_container}@{workspace_default_storage_account}.blob.core.windows.net/"
)


StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 9, Finished, Available)

In [4]:
comments_path = "/basketball_comments_sep_v2.parquet"
submissions_path = "/basketball_submissions_sep.parquet"

comments = spark.read.parquet(f"{workspace_wasbs_base_url}{comments_path}")
submissions = spark.read.parquet(f"{workspace_wasbs_base_url}{submissions_path}")

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 10, Finished, Available)

### Exploring the Data

In [5]:
submissions = submissions.cache()
comments = comments.cache()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 11, Finished, Available)

#### Data Shape and Schema

The breakdown of the 3 subreddits by number of posts is heavily imbalanced as is expected. In submissions there are just over 22,000 posts and in comments there are just over 1,575,000 comments. The majority of these posts come from the CollegeBasketball subreddit. 

In [8]:
print("Count of Submissions by Subreddit")
print(submissions.groupBy('subreddit').count().show())

print("Count of Comments by Subreddit")
print(comments.groupBy('subreddit').count().show())

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 13, Finished, Available)

Count of Submissions by Subreddit
+-----------------+-----+
|        subreddit|count|
+-----------------+-----+
|         jayhawks|  632|
|CollegeBasketball|21290|
|         tarheels|  230|
+-----------------+-----+

None
Count of Comments by Subreddit
+-----------------+-------+
|        subreddit|  count|
+-----------------+-------+
|         jayhawks|   6541|
|CollegeBasketball|1567801|
|         tarheels|    828|
+-----------------+-------+

None


The important variables in the submissions dataframe are 

1. `subreddit`: The Subreddit that the post is in
2. `author` : Who authored the post
3. `selftext` : The contents of the post
4. `created_utc` : The timestamp of the post

The important variables in the comments dataframe are 

1. `subreddit`: The Subreddit that the post is in
2. `author` : Who authored the post
3. `body` : The contents of the post
4. `created_utc` : The timestamp of the post

In [6]:
print("Submissions Schema")
submissions.printSchema()

print("Comments Schema")
comments.printSchema()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 12, Finished, Available)

Submissions Schema
root
 |-- subreddit: string (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- title: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- num_comments: long (nullable = true)

Comments Schema
root
 |-- subreddit: string (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- score: long (nullable = true)



#### Missing Data

Since Reddit is a user driven platform there will likely be issues with missing text in posts or deleted posts/comments. Lets check for missing data before we proceed.

In [7]:
# Submissions
missing_values = ["", "[deleted]", "[removed]"]
missing_df = submissions.withColumn("selftext", when(col("selftext").isin(missing_values), None).otherwise(col("selftext")))
missing_data_count = missing_df.filter(col("selftext").isNull()).count()
print("Missing Text Column in Submissions =",missing_data_count)
# Comments
missing_df_c = comments.withColumn("body", when(col("body").isin(missing_values), None).otherwise(col("body")))
missing_data_count = missing_df_c.filter(col("body").isNull()).count()
print("Missing Text Column in Comments =",missing_data_count)

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 13, Finished, Available)

Missing Text Column in Submissions = 12348
Missing Text Column in Comments = 85295


There are over 12,000 missing posts in `Submissions` and around 85,000 missing comments in `Comments`. We will remove these before doing further analysis

In [8]:
submissions = missing_df.na.drop(subset=["selftext"])
comments = missing_df_c.na.drop(subset = ["body"])

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 14, Finished, Available)

The new counts of the subreddits with missing removed show about half of the submission data remaining, but a large portion of the comments data is intact.

In [9]:
print("Count of Submissions by Subreddit with Missing Removed")
print(submissions.groupBy('subreddit').count().show())

print("Count of Comments by Subreddit with Missing Removed")
print(comments.groupBy('subreddit').count().show())

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 15, Finished, Available)

Count of Submissions by Subreddit with Missing Removed
+-----------------+-----+
|        subreddit|count|
+-----------------+-----+
|         jayhawks|  204|
|CollegeBasketball| 9567|
|         tarheels|   33|
+-----------------+-----+

None
Count of Comments by Subreddit with Missing Removed
+-----------------+-------+
|        subreddit|  count|
+-----------------+-------+
|         jayhawks|   6328|
|CollegeBasketball|1482767|
|         tarheels|    780|
+-----------------+-------+

None


#### Adding Day, Month, Year Columns

In [10]:
submissions = (
    submissions
    .withColumn("day", dayofmonth(col("created_utc")))
    .withColumn("month", month(col("created_utc")))
    .withColumn("year", year(col("created_utc")))
)
comments = (
    comments
    .withColumn("day", dayofmonth(col("created_utc")))
    .withColumn("month", month(col("created_utc")))
    .withColumn("year", year(col("created_utc")))
)

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 16, Finished, Available)

#### First Snapshot of Data

Here we can see the most important columns from `submissions` and `comments`. The `selftext` and `body` columns hold the post and comments, respectively.

In [11]:
print("Submission Data")
submissions.select("subreddit", "author", "title", "selftext", "created_utc", "num_comments").show()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 17, Finished, Available)

Submission Data
+-----------------+--------------------+--------------------+--------------------+-------------------+------------+
|        subreddit|              author|               title|            selftext|        created_utc|num_comments|
+-----------------+--------------------+--------------------+--------------------+-------------------+------------+
|CollegeBasketball|             rCBBMod|Announcement: The...|Hello r/CollegeBa...|2022-03-17 14:00:50|           9|
|CollegeBasketball|       AutoModerator|[Weekly Post] Fre...|Are you new to th...|2022-04-29 15:00:11|           0|
|CollegeBasketball|       AutoModerator|[Weekly Post] Fre...|It's Free Talk Fr...|2022-04-29 15:00:11|           1|
|CollegeBasketball|              cbbBot|[Game Thread] Neb...|[Nebraska](#f/neb...|2022-02-23 00:54:05|          41|
|CollegeBasketball|              cbbBot|[Game Thread] Kan...|[Kansas State](#f...|2022-02-23 01:00:04|         626|
|CollegeBasketball|              cbbBot|[Game Thread] #2

In [12]:
print("Comments Data")
comments.select("subreddit", "author","author_flair_text", "body", "created_utc", "score").show()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 42, 18, Finished, Available)

Comments Data
+-----------------+-------------------+--------------------+--------------------+-------------------+-----+
|        subreddit|             author|   author_flair_text|                body|        created_utc|score|
+-----------------+-------------------+--------------------+--------------------+-------------------+-----+
|CollegeBasketball|         joeveralls|:cincinnati: :ohi...|Nothing is fallin...|2021-11-14 00:27:16|    5|
|CollegeBasketball|       DaddyStone13|:cincinnati: :big...|lol i forgot abou...|2021-11-14 00:27:51|    4|
|CollegeBasketball|        seahawksjoe|   :usc: USC Trojans|Also, half of the...|2021-11-14 00:28:36|    4|
|CollegeBasketball|       CheeseAndCam|:iowastate: Iowa ...|I like how Creigh...|2021-11-14 00:30:06|    2|
|CollegeBasketball|          [deleted]|                null|  Losing, apparently|2021-11-14 00:30:17|   14|
|CollegeBasketball|       corranhorn57|:cincinnati: Cinc...|Oklahoma lost, th...|2021-11-14 00:30:26|    5|
|CollegeBasket

#### Creating author flair column

`author_flair_text` is a column that contains information about the user icon of the reddit user. In most of these posts this is the team that the author supports. Here we create a new variable that indicates who the author of the comment or post supports

In [9]:
team_pattern = r":([\w\s]+):"

# Extract the team names and create the "supported_team" column
submissions = submissions.withColumn("supported_team", regexp_extract(col("author_flair_text"), team_pattern, 1))
comments = comments.withColumn("supported_team", regexp_extract(col("author_flair_text"), team_pattern, 1))

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 16, 14, Finished, Available)

If we remove the `r/tarheels` and `r/jayhawks` subreddits from our data temporarily we can get a good idea of what teams are the most supported in the user base of `r/CollegeBasektball`. 

In `submissions` the most common icons are the default `rcbb` (r/collegebasketball) and `null` (no icon). Ignoring those we see that the most supported teams are Florida State, Illinois, Purdue, and Duke. In `comments`, `null` is the most common icon, however after that the most supported teams are Purdue, North Carolina, Illinois, and Kentucky. 

This shows that some fan bases may be more active posters while others may be more involved in the comments section. 

In [39]:
print("Submissions Top Supported Teams")
submissions.filter(col("subreddit")=="CollegeBasketball").groupBy("supported_team").agg(count("*").alias("count")).orderBy(desc("count")).show()
print("Comments Top Supported Teams")
comments.filter(col("subreddit")=="CollegeBasketball").groupBy("supported_team").agg(count("*").alias("count")).orderBy(desc("count")).show()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 14, 44, Finished, Available)

Submissions Top Supported Teams
+----------------+-----+
|  supported_team|count|
+----------------+-----+
|            rcbb| 6632|
|            null|  821|
|    floridastate|  124|
|        illinois|   89|
|          purdue|   84|
|            duke|   69|
|   northcarolina|   66|
|          kansas|   64|
|       iowastate|   64|
|        virginia|   62|
|        kentucky|   62|
|     connecticut|   51|
|          dayton|   48|
|        michigan|   43|
|mississippistate|   42|
|         indiana|   36|
|         gonzaga|   35|
|    sanjosestate|   35|
|   michiganstate|   33|
|          auburn|   32|
+----------------+-----+
only showing top 20 rows

Comments Top Supported Teams
+--------------+------+
|supported_team| count|
+--------------+------+
|          null|173474|
|        purdue| 72772|
| northcarolina| 68933|
|      illinois| 62610|
|      kentucky| 56856|
|        kansas| 55664|
|      michigan| 52395|
| michiganstate| 51169|
|       indiana| 40537|
|        auburn| 39623|
|

#### Creating Game Thread Indicator and details about game (teams, ranking, time)

Some posts in the submissions of `r/CollegeBasketball` are game threads where people can comment on and discuss specific games that are being played. We can extract information about the title to determine the teams playing in the game, their rankings, and the time the game is being played. 

From here we can determine which games have the most comments and thereby can act as a proxy for which games were the most watched on reddit. We see that the most commented on games involve North Carolina, and further analysis of these games show that the three most popular games are the Elite Eights, Final Four, and Championship games for North Carolina. Additionally we see that all of the most popular games are between ranked teams. 

Expanding on this finding that ranked matchups seem to have more interest we did some further analysis to look at the popularity of games over the course of a season and also the difference between ranked and unranked mathcups. Here we see that ranked matchups have significantly higher average number of comment in all months of the season. If we compare only February, ranked matchups have on average 12.5 times the number of comments of unranked matchups.  Additionally we see that April and March have the highest number of average comments across both groups, while the earlier months in the season (November and December) have lower average comments per game. 

In [57]:
# create condition for title column starting with the string [Game Thread]
condition = submissions["title"].startswith("[Game Thread]")
# create new column in submissions indicating if post is a game thread
submissions = submissions.withColumn("game_thread", when(condition, True).otherwise(False))

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 16, 62, Finished, Available)

In [58]:
teams_pattern = "\[Game Thread\] (#?(\d*) )?([^\[@]+) @ (#?(\d*) )?([^\[@]+) \((\d{2}:\d{2} [APMapm]{2} [A-Za-z]+)\)"
submissions = submissions.withColumn("home_ranking", regexp_extract(col("title"), teams_pattern, 5))
submissions = submissions.withColumn("home_team", regexp_extract(col("title"), teams_pattern, 6))
submissions = submissions.withColumn("away_ranking", regexp_extract(col("title"), teams_pattern, 2))
submissions = submissions.withColumn("away_team", regexp_extract(col("title"), teams_pattern, 3))
submissions = submissions.withColumn("game_time", regexp_extract(col("title"), teams_pattern, 7))
submissions = submissions.withColumn("ranked_matchup", when((col("home_ranking")!="")& (col("away_ranking")!=""),True).otherwise(False))

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 16, 63, Finished, Available)

In [32]:
submissions.filter(col("game_thread")==1).orderBy(col("num_comments").desc()).limit(20).select("title", "num_comments").show(truncate = False)

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 16, 37, Finished, Available)

+-----------------------------------------------------------------+------------+
|title                                                            |num_comments|
+-----------------------------------------------------------------+------------+
|[Game Thread] #8 North Carolina @ #1 Kansas (09:20 PM ET)        |20723       |
|[Game Thread] #8 North Carolina @ #1 Baylor (12:10 PM ET)        |18640       |
|[Game Thread] #8 North Carolina @ #2 Duke (08:49 PM ET)          |11959       |
|[Game Thread] #15 Saint Peter's @ #3 Purdue (07:09 PM ET)        |9872        |
|[Game Thread] #4 Arkansas @ #1 Gonzaga (07:09 PM ET)             |9648        |
|[Game Thread] #15 Saint Peter's @ #8 North Carolina (05:05 PM ET)|8080        |
|[Game Thread] #9 TCU @ #1 Arizona (09:40 PM ET)                  |7972        |
|[Game Thread] #3 Texas Tech @ #2 Duke (09:39 PM ET)              |7045        |
|[Game Thread] #7 Michigan State @ #2 Duke (05:15 PM ET)          |6857        |
|[Game Thread] #4 Arkansas @

In [60]:
submissions.filter(col("game_thread")==1).groupBy("year", "month", "ranked_matchup").agg(round(mean("num_comments"),0).alias("average_comments"), sum(col("num_comments")).alias("total_comments")).orderBy(col("average_comments").desc()).limit(11).show()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 16, 65, Finished, Available)

+----+-----+--------------+----------------+--------------+
|year|month|ranked_matchup|average_comments|total_comments|
+----+-----+--------------+----------------+--------------+
|2022|    4|          true|         12685.0|         38055|
|2022|    3|          true|          3290.0|        263187|
|2021|   11|          true|          1818.0|         18181|
|2022|    2|          true|          1570.0|         48671|
|2022|    1|          true|          1470.0|         38213|
|2021|   12|          true|           740.0|          7395|
|2022|    3|         false|           215.0|         97438|
|2022|    1|         false|           129.0|         84812|
|2022|    2|         false|           125.0|         93661|
|2021|   12|         false|           103.0|         54828|
|2021|   11|         false|            89.0|         58010|
+----+-----+--------------+----------------+--------------+



Using this cleaned data we will proceed to answer our business questions

In [ ]:
comments.show()

In [ ]:
comments.write.mode("overwrite").parquet(f"{workspace_wasbs_base_url}/basketball_comments_clean_v2.parquet")